In [1]:
#   %pip install pandas
%pip install bcrypt


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
import os
import argparse
import csv
import json
import pandas as pd
import numpy as np


In [3]:
already_created_dst_file = True
#   - anime : [animeID, (count_reviews), avg_reviews ] 
#   - anime : details [animeID, episodes, ... ]

if already_created_dst_file is not True :
    #   read current file
    filename = './000_unpolished_csv/users_cleaned.csv'
    firaw = pd.read_csv(filename, nrows= 10)
    print(f'head : {firaw.columns.values}')
    #   head : ['username' 'user_id' 'user_watching' 'user_completed' 'user_onhold'
    #            'user_dropped' 'user_plantowatch' 'user_days_spent_watching' 'gender'
    #            'location' 'birth_date' 'access_rank' 'join_date' 'last_online'
    #            'stats_mean_score' 'stats_rewatched' 'stats_episodes']
    
    f_user_details = './final_csv/user_details.csv'
    if not os.path.exists(f_user_details):
        with open(f_user_details, 'w'): pass   

#   copy related only if type is not manga

    head : ['username' 'user_id' 'user_watching' 'user_completed' 'user_onhold'
     'user_dropped' 'user_plantowatch' 'user_days_spent_watching' 'gender'
     'location' 'birth_date' 'access_rank' 'join_date' 'last_online'
     'stats_mean_score' 'stats_rewatched' 'stats_episodes']
    
    user_id                                 2255153
    username                               karthiga
    user_watching                                 3
    user_completed                               49
    user_onhold                                   1
    user_dropped                                  0
    user_plantowatch                              0
    user_days_spent_watching              55.091667
    gender                                   Female
    birth_date                  1990-04-29 00:00:00
    access_rank                                 NaN
    join_date                   2013-03-03 00:00:00
    stats_mean_score                           7.43
    stats_rewatched                             0.0
    stats_episodes                             3391
    Name: 0, dtype: object

In [1]:
import sys
import os
import argparse
import csv
import json
import pandas as pd
import numpy as np

#   add email, password

#   #   #   old_list_user_details = [ 
#   #   #                   'user_id' , 
#   #   #                   'username' , 
#   #   #                   'user_watching' , 
#   #   #                   'user_completed' , 
#   #   #                   'user_onhold' , 
#   #   #                   'user_dropped' , 
#   #   #                   'user_plantowatch' , 
#   #   #                   'user_days_spent_watching' , 
#   #   #                   'gender' , 
#   #   #                   'birth_date' , 
#   #   #                   'access_rank' , 
#   #   #                   'join_date' , 
#   #   #                   'stats_rewatched' , 
#   #   #                   'stats_episodes'  
#   #   #                   ]

list_user_details = [ 
                    'user_id'  
                ,   'username'  
                ,   'gender'  
                ,   'birth_date'  
                ,   'join_date'  
                ,   'stats_episodes'  
            ]

expanded_list_user_details = [ 
                    'user_id'  
                ,   'username'  
                ,   'name'              #   from rng gen lists
                ,   'surname'           #   from rng gen lists
                ,   'email' 
                ,   'password' 
                ,   'gender'  
                ,   'birth_date'  
                ,   'join_date'  
                ,   'stats_episodes'  
                ,   'most_recent_reviews'
            ]

#   print(f'lenght of target cols : {len(list_user_details)}')

import time

t0 = time.time_ns()

import hashlib
import bcrypt
import uuid
import secrets
def hash_salt(password) :
    #   this_salt = bcrypt.gensalt()
    #   hashed_pwd = hashlib.pbkdf2_hmac(hash_name='sha256', password=password.encode('utf-8'), salt=this_salt, iterations=31)
    #   hashed = bcrypt.hashpw(password, this_salt)
    #   salt = secrets.token_bytes(16).decode('utf-16')
    salt = uuid.uuid4().hex
    #   print(f' salt {salt} type : {type(salt)} ___ password {password} type : {type(password)}')
    concat = str(salt+password)
    #   hashed = hashlib.sha512( salt + password ).hexdigest()
    hashed = hashlib.sha256( concat.encode() ).hexdigest() 
    return [ hashed , salt ]



transfer_data = True

if transfer_data == True:
    # read from source
    src = './000_unpolished_csv/users_cleaned.csv'


    file_base = pd.read_csv(src)
    extracted = file_base[list_user_details]
    #   print(extracted.columns.values.size)

    extracted = extracted[extracted['username'].notna()]

    num_rows = len(extracted.index)
    
    t1 = time.time_ns()
    print(f'time to load file : {t1-t0} nanoseconds')

    # now we need to add email e password
    extracted['email'] = extracted.apply( lambda row: f'{row['username']}@gmail.com' , axis=1 )
    extracted['password'] = extracted.apply( 
                                #   lambda row: f'{row['username']}{row['birth_date'].split(' ')[0].replace('-','')}' 
                                    lambda row: hash_salt(row['username'])
                                , axis=1 ) 
    
    # now we need to add name and surname
    file_nam_sur = './random_names.txt'
    nslist = []
    countline = 0
    with open(file_nam_sur, 'rb') as nasufile :
        for line in nasufile:
            #   print(f'line : {line} _ type : {type(line)}')
            nam, sur =  line.decode('utf-8').split(' ') 
            nam = nam.replace('\n', '')
            sur = sur.replace('\n', '')
            nslist.append( [nam, sur] )
            countline +=1
            if countline >= num_rows :
                break
    if countline < num_rows :
        print(f'DBG->>> need more names !!!!!!')
        exit()
    #   num_new_names = len(datanamsur.index)
    
    datanamsur = pd.DataFrame(nslist, columns=['name','surname'] )

    extracted['name'] = datanamsur['name'].values

    extracted['surname'] = datanamsur['surname'].values

    extracted['most_recent_reviews'] = np.empty((len(extracted), 0)).tolist()


    # now we reorder it !
    extracted = extracted.reindex(columns=expanded_list_user_details)
    print(extracted.iloc[0])
    

    t2 = time.time_ns()
    print(f'time to edit file : {t2-t1} nanoseconds')

    #   #   extract to destination
    dstcsv = './final_csv/user_details.csv'
    if not os.path.exists(dstcsv):
        with open(dstcsv, 'w'): pass   
    extracted.to_csv(dstcsv, index=False)


    dstjson = './final_csv/user_details.json'
    if not os.path.exists(dstjson):
        with open(dstjson, 'w'): pass   
    extracted.to_json(dstjson, index=False)
    


    t3 = time.time_ns()
    print(f'time to save file : {t3-t2} nanoseconds')


#   print output :  #   (success!!!)
#   #   time to load file : 290000700 nanoseconds
#   #   user_id                                                          2255153
#   #   username                                                        karthiga
#   #   name                                                               Emery
#   #   surname                                                             Kirk
#   #   email                                                 karthiga@gmail.com
#   #   password               [f5cc5b00aa87bab4e1e9655debbf7744878f2e444888f...
#   #   gender                                                            Female
#   #   birth_date                                           1990-04-29 00:00:00
#   #   join_date                                            2013-03-03 00:00:00
#   #   stats_episodes                                                      3391
#   #   most_recent_reviews                                                   []
#   #   Name: 0, dtype: object
#   #   time to edit file : 1477028600 nanoseconds
#   #   time to save file : 0 nanoseconds   

#   

time to load file : 287028600 nanoseconds
user_id                                                          2255153
username                                                        karthiga
name                                                               Emery
surname                                                             Kirk
email                                                 karthiga@gmail.com
password               [6ac0da893e64d4944dd3b81f0beaae632b77b8ec5127f...
gender                                                            Female
birth_date                                           1990-04-29 00:00:00
join_date                                            2013-03-03 00:00:00
stats_episodes                                                      3391
most_recent_reviews                                                   []
Name: 0, dtype: object
time to edit file : 1445999400 nanoseconds
time to save file : 674971600 nanoseconds


In [3]:
################################################################
import ast
import sys
import os
import argparse
import csv
import json
import pandas as pd
import numpy as np

#   DO RELATIONS GRAPH :

#   anime_id_src < anime_id_dst

anime_namid = [
            'user_id'
        ,   'username'
    ]



do_useridusername = True

if do_useridusername == True:
    import pandas as pd
    import ast

    #   #   READ FROM SOURCE
    
    src = './000_unpolished_csv/users_cleaned.csv'
    trgcols = [
                'user_id'
            ,   'username'
        ]

    
    user_file = pd.read_csv(src, usecols=trgcols)

    print(f'src file keys : {user_file.keys()}')

    #   #       #   #   LIST OF LIST TO DATAFRAME :
    #   #       datagenres = pd.DataFrame(genres_chart , columns= genres_cols)


    #   SAVE TO JSON :
    dst_userjson = './final_csv/userid_username_graph.json'
    if not os.path.exists(dst_userjson):
        with open(dst_userjson, 'w'): pass   
    user_file.to_json(dst_userjson, index=False)
 
  
    #   SAVE TO CSV :
    dst_usercsv = './final_csv/userid_username_graph.csv'
    if not os.path.exists(dst_usercsv):
        with open(dst_usercsv, 'w'): pass   
    user_file.to_csv(dst_usercsv, index=False)








src file keys : Index(['username', 'user_id'], dtype='object')
